In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.integrate import quad
from math import erf, sqrt

# Load the data from the file
file_path = r"C:\Users\David1\Downloads\ciul.mca"

# Read the file as a binary to understand its structure
with open(file_path, 'rb') as file:
    file_content = file.read().decode('latin1', errors='ignore')

# Locate the start of the data
data_start_index = file_content.find('<<DATA>>') + len('<<DATA>>')
data_string_full = file_content[data_start_index:]

# Split the data into individual counts
counts_full = [int(value) for value in data_string_full.split() if value.isdigit()]

# Trim the data to match exactly 8192 channels
counts_trimmed = counts_full[:8192]

# Define the cut range
cut_start = 6200  # Start of the cut (channel number)
cut_end = 6500    # End of the cut (channel number)

# Apply the cut to the data
x_values_cut = np.arange(cut_start, cut_end)
counts_cut = counts_trimmed[cut_start:cut_end]

# Define a Gaussian function for fitting
def gaussian(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

# Estimate initial parameters for fitting
a_initial = max(counts_cut)
x0_initial = cut_start + np.argmax(counts_cut)
sigma_initial = 50  # somewhat arbitrary guess

initial_params = [a_initial, x0_initial, sigma_initial]

# Curve fitting
fit_successful = False
try:
    popt, pcov = curve_fit(
        gaussian, 
        x_values_cut, 
        counts_cut, 
        p0=initial_params, 
        maxfev=5000,
        bounds=([0, cut_start, 0], [np.inf, cut_end, np.inf])
    )
    perr = np.sqrt(np.diag(pcov))  # standard errors of a, x0, sigma
    
    print("Fitted parameters:")
    print(f"a     = {popt[0]:.4g} ± {perr[0]:.4g}")
    print(f"x0    = {popt[1]:.4g} ± {perr[1]:.4g}")
    print(f"sigma = {popt[2]:.4g} ± {perr[2]:.4g}")
    fit_successful = True
except RuntimeError as e:
    print("Fit did not converge:", e)

# ---------------------------------------------------------------------
# 1. Numerical area from best-fit function (via quad):
# ---------------------------------------------------------------------
if fit_successful:
    def fitted_gaussian(x):
        return popt[0] * np.exp(-(x - popt[1])**2 / (2 * popt[2]**2))

    area_quad, area_error_quad = quad(fitted_gaussian, cut_start, cut_end)

    print(f"\nNumeric integral (quad) of fitted Gaussian from {cut_start} to {cut_end}:")
    print(f"  Area = {area_quad:.4g} ± {area_error_quad:.4g} (integration numerical uncertainty)")

# ---------------------------------------------------------------------
# 2. Theoretical area from closed-form expression + error propagation
# ---------------------------------------------------------------------
if fit_successful:
    a_fit, x0_fit, sigma_fit = popt
    # closed-form of area from x1 to x2
    def truncated_gaussian_area(a, x0, sigma, x1, x2):
        return a * np.sqrt(np.pi/2) * sigma * (
            erf((x2 - x0)/(np.sqrt(2)*sigma)) - erf((x1 - x0)/(np.sqrt(2)*sigma))
        )

    # partial derivatives of I(a, x0, sigma) w.r.t. a, x0, sigma
    def dI_da(x1, x2, a, x0, sigma):
        """d/d(a) of area I."""
        # basically the same as I but without the factor 'a'
        return np.sqrt(np.pi/2) * sigma * (
            erf((x2 - x0)/(np.sqrt(2)*sigma)) - erf((x1 - x0)/(np.sqrt(2)*sigma))
        )

    def dI_dx0(x1, x2, a, x0, sigma):
        """d/d(x0) of area I."""
        # derivative of erf(...) w.r.t x0 is
        #   d/dx0 erf((x - x0)/(sqrt(2)*sigma)) = - (1/(sqrt(2)*sigma)) * 2/sqrt(pi)*exp(-u^2)
        # but let's do it by difference of x2 and x1 terms:
        factor = np.sqrt(np.pi/2) * a * sigma
        # define helper for d/dx0 of erf(...) 
        def d_erf_dx0(x):
            u = (x - x0)/(np.sqrt(2)*sigma)
            return - (1.0/(np.sqrt(2)*sigma)) * (2.0/np.sqrt(np.pi)) * np.exp(-u**2)
        return factor * (d_erf_dx0(x2) - d_erf_dx0(x1))

    def dI_dsigma(x1, x2, a, x0, sigma):
        """d/d(sigma) of area I."""
        # I(a, x0, sigma) = a * sqrt(pi/2) * sigma * [erf(...)-erf(...)]
        # So there's:
        #    1) derivative w.r.t. sigma of the prefactor sigma
        #    2) derivative w.r.t. sigma inside the erf(...) argument
        from math import exp
        # piece1: derivative of (a * sqrt(pi/2) * sigma) = a * sqrt(pi/2)
        piece1 = a * np.sqrt(np.pi/2) * (
            erf((x2 - x0)/(np.sqrt(2)*sigma)) - erf((x1 - x0)/(np.sqrt(2)*sigma))
        )
        # piece2: derivative of the erf(...) difference w.r.t. sigma
        #   d/dsigma erf((x - x0)/(sqrt(2)*sigma)) = derivative wrt 'u' times derivative of u wrt sigma
        #   u = (x - x0)/(sqrt(2)*sigma)
        #   du/dsigma = -(x - x0)/(sqrt(2)*sigma^2)
        def d_erf_dsigma(x):
            u = (x - x0)/(np.sqrt(2)*sigma)
            # d/du erf(u) = 2/sqrt(pi)*exp(-u^2)
            return (2.0/np.sqrt(np.pi)) * np.exp(-u**2) * (-(x - x0)/(np.sqrt(2)*sigma**2))
        piece2 = a * np.sqrt(np.pi/2) * sigma * ( d_erf_dsigma(x2) - d_erf_dsigma(x1) )
        return piece1 + piece2

    # Evaluate closed-form area at best-fit parameters
    area_cf = truncated_gaussian_area(a_fit, x0_fit, sigma_fit, cut_start, cut_end)

    # Construct gradient = [∂I/∂a, ∂I/∂x0, ∂I/∂σ]
    grad = np.array([
        dI_da(cut_start, cut_end, a_fit, x0_fit, sigma_fit),
        dI_dx0(cut_start, cut_end, a_fit, x0_fit, sigma_fit),
        dI_dsigma(cut_start, cut_end, a_fit, x0_fit, sigma_fit),
    ])

    # Error propagation: Var(I) = grad^T * pcov * grad
    # pcov is the 3x3 covariance matrix of (a, x0, sigma)
    var_I = grad @ pcov @ grad
    # If var_I < 0 due to numerical issues, clip to zero
    var_I = max(var_I, 0.0)
    std_I = np.sqrt(var_I)

    print(f"\nClosed-form integral of the Gaussian from {cut_start} to {cut_end}:")
    print(f"  Area = {area_cf:.4g}")
    print(f"  Theoretical error from fit params = ±{std_I:.4g}")

# Plot the cut data and the fitted Gaussian
plt.figure(figsize=(10, 6))
plt.plot(x_values_cut, counts_cut, label='Counts per Channel (Cut)')
if fit_successful:
    plt.plot(x_values_cut, gaussian(x_values_cut, *popt), 'r-', label='Gaussian Fit')
plt.xlabel('Channel Number')
plt.ylabel('Counts')
plt.title('Counts per Channel')
plt.legend()
plt.grid(True)
plt.show()

